In [18]:
import numpy as np 
import composite as cp 

In [60]:
class compound:
    def __init__(self,layer_list,status='S'):
        if status=='S':
            layout=np.array(layer_list)
            reverse=layout[::-1]
            self.layout=np.append(layout,reverse)
            pass
        else:
            self.layout=np.array(layer_list)
            pass
        self.__get_t()
        self.__get_angle()
        self.h=np.sum(self.t)
        self.__ABDcal()
        pass

    def __get_t(self):
        t=np.array([])
        for item in self.layout:
            t=np.append(t,item.t)
            self.t=t
            pass
        pass

    def __get_angle(self):
        angle=np.array([])
        for item in self.layout:
            angle=np.append(angle,item.angle)
            self.angle=angle
            pass
        pass

    def __ABDcal(self):
        h_temp=0
        A,B,D=0,0,0
        for item in self.layout:
            if h_temp<self.h:
                bottom=h_temp-self.h/2
                head=bottom+item.t
                A=A+item.Q_*np.abs(head-bottom)
                B=B+item.Q_*np.abs(head**2-bottom**2)/2
                D=D+item.Q_*np.abs(head**3-bottom**3)/3
                h_temp=h_temp+item.t 
                pass
            else:
                break
            pass
        self.A=A
        self.B=B
        self.D=D 
        pass


In [61]:
A = cp.layer('HT3/5224')
A.set_angle_degree(45)
B = cp.layer('HT3/5224')
B.set_angle_degree(-45)
C = cp.layer('HT3/5224')
C.set_angle_degree(0)

comp = compound([A,B,C],status='S')
comp.h
comp.t
comp.A

array([[57239414.09103504, 17732247.66366911,        0.        ],
       [17732247.66366911, 24140343.58548074,        0.        ],
       [       0.        ,        0.        , 19207631.17458878]])